# Build a Simple LLM Applicatin with LCEL

- LCEL : LangChain Expression Language
- used to chain components together.

In [3]:
# Load environment variables and create LLAMA2 model instance
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOllama

load_dotenv()

model = ChatOllama(model="llama2")

In [6]:
# ChatModels are instances of Langchain runnables meaning they expose a standard interface to interact with them
# Provide context to the LLM which can be a list of past conversation between LLM and user and LLM will generate 
# output keeping past conversation in mind 
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='\nCiao!', response_metadata={'model': 'llama2', 'created_at': '2024-07-15T08:02:25.3010567Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3266497500, 'load_duration': 20996200, 'prompt_eval_count': 30, 'prompt_eval_duration': 2023174000, 'eval_count': 5, 'eval_duration': 1218035000}, id='run-9d4584ac-7d5c-4a8d-a09d-89da230b2652-0')

In [8]:
# LLMs response is an AIMessage object, when we want to work with plain strings, we can use below parser
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

In [9]:
# More commonly, we can "chain" the model with this output parser. 
# This means this output parser will get called every time in this chain. 
# This chain takes on the input type of the language model (string or list of message) and 
# returns the output type of the output parser (string).

chain = model | parser
chain.invoke(messages)

'Ciao!'

In [10]:
# The ChatPromptTemplate in LangChain is a way to create structured conversation prompts for chatbots or conversational agents.
from langchain_core.prompts import ChatPromptTemplate
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

In [11]:
# Read the messages as list
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

In [13]:
# Now chain together the prompt template with LLM and parser
chain = prompt_template | model | parser
chain.invoke({"language": "japanese", "text": "hi"})

'\nこんにちは (Konnichiwa)'

In [14]:
# Run this code if we have the serve.py running where we wrote the chain to interact with LLM
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'Italian: Ciao'